In [ ]:
import math
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans, SpectralClustering

In [ ]:
class Node(object):
    def __init__(self,number,neighbors,degree):
        self.name = number
        self.neighbors = neighbors
        self.degree = degree
        self.data = None                       #Holds the local data Pi
        self.centers = None                    #Holds the centers Bi
        self.local_coreset = None              #To store coreset, i.e. Si U Ai
        self.weights = None                    #To store the weight of points in local coreset Si U Ai
        self.message_received = {}
        self.X = None                          #To store the final centers
    def set_data(self,data):
        self.data = data
    def set_centers(self,centers):
        self.centers = centers
    def set_local_coreset(self,S):
        self.local_coreset = S
    def set_weights(self,weights):
        self.weights = weights
    def set_X(self,X):
        self.X = X
        
communication_cost = 0

In [ ]:
def create_random_graph(no_of_nodes,probability):
    return nx.erdos_renyi_graph(no_of_nodes,probability)

def create_preferential_graph(n,m):
    # n = number of nodes, m = number of edges 
    return nx.generators.random_graphs.barabasi_albert_graph(n,m)

def create_grid_graph(n,m):
    return nx.grid_2d_graph(n,m)

In [ ]:
#Function to get sequence of iterating the nodes
#Returns 'nodes' used in fuction arguements later in message passing
def node_sequence(G):
    seq=[]
    l = list(nx.dfs_edges(G,0))
    for i in range(len(l)-1):
        if (l[i][1] == l[i+1][0]):
            seq.append(l[i][0])
        else:
            seq.append(l[i][0])
            seq.append(l[i][1])
            p = nx.shortest_path(G,l[i][1],l[i+1][0])
            for k in range(1,len(p)-1):
                seq.append(p[k])
    seq.append(l[-1][0])
    seq.append(l[-1][1])
    return seq

In [ ]:
#Incomplete
def uniform_partitioning(df,nodes):
    temp_df = df.copy(deep=True)
    size_of_pi = math.floor(df.shape[0]/len(nodes))
    for node in nodes:
        if node != nodes[-1]:
            node.data = temp_df.sample(size_of_pi)
            temp_df.drop(node.data.index,inplace = True)
        else:
            node.data = temp_df
    return
            
def similarity_partitioning(df,nodes):
    temp_df = df.copy(deep=True)
    spec=SpectralClustering(n_clusters=len(nodes), gamma=1.0)
    c_id = spec.fit_predict(temp_df)
    for i in range(len(nodes)):
        nodes[i].data = temp_df[c_id==i]      
    return

def weighted_partitioning(df,nodes):
    #code here
    return

def degree_partitioning(df,nodes):
    #code here
    return

In [ ]:
def clustering_algo( data, no_of_centers ):
    kmeans = KMeans(n_clusters=no_of_centers, init = 'random', random_state=0).fit(np.array(data))
    #init stands for initialization, i.e. how to get first set of centers. Default is k-means++. I went for random
    #random_state stands for the seed to generate random centers in the beginning. Kept it zero to get same centers everytime.
    #Remove if not required
    #The package uses LLoyd's algorithm
    return pd.DataFrame(kmeans.cluster_centers_)      #this will contain a dataframe of centers, was an np array


In [ ]:
def Message_Passing(message,neighbors,node):
    global communication_cost
    if node not in node.message_received: 
        node.message_received[node] = message
    for neighbor in neighbors:
        for message in node.message_received.values():
            if node not in neighbor.message:
                communication_cost+=1
                neighbor.message_received[node] = message
    return

In [ ]:
def get_cost( data, centers):
    distanceMatrix = pd.DataFrame(np.nan * np.ones(shape=(data.shape[0],centers.shape[0])))
    for j in range(0, centers.shape[0]):
        for i in range(0, data.shape[0]):
            distanceMatrix.iloc[i][j] = sqrt((centers.iloc[j][0] – data.iloc[i][0])**2 + (centers.iloc[j][1] – data.iloc[i][1])**2)
    #print (distanceMatrix)
    return distanceMatrix.min(axis=1)

def distributed_coreset_construction( nodes, t, no_of_centers ):
    for node in nodes:
        node.centers = clustering_algo(node.data, no_of_centers)    #this will contain centers stored in dataframe
        cost_of_each_data = get_cost( node.data, node.centers)  #comes back as pandas series
        Message_Passing(cost_of_each_data.sum(),node.neighbors,node)
    for node in nodes.reverse():
        cost_of_each_data = get_cost( node.data, node.centers)  #comes back as pandas series
        Message_Passing(cost_of_each_data.sum(),node.neighbors,node)
    for node in nodes:
        cost_of_each_data = get_cost( node.data, node.centers)  #comes back as pandas series
        t_i = (t*node.message_received[node])/sum(node.message_received.values())
        m_p = 2*cost_of_each_data
        S_i = node.data.sample(n=t_i,weights=m_p)
        w_q = sum(node.cost_of_nodes.values())/(t*m_p[S_i.index])
        w_b = []
        for index, b in node.centers.iterrows():
            Pb = node.data[np.sqrt(np.square(np.array(node.data) - np.array(b)).sum(1)) == cost_of_each_data]  
            #previous line measures euc dist of each point with center b and compares the values with min cost i.e.min d(p,X)over all x belonging to X
            #Pb will be a dataframe
            w_b.append(Pb.shape[0] - sum([w_q[x] for x in list(map(lambda x:S_i.index.get_loc(x),S_i.index.intersection(Pb.index)))]))
        node.set_local_coreset(pd.concat([S_i,node.centers]))
        node.set_weight(w_q.extend(w_b))
    return


In [ ]:
def distributed_clustering_on_graph(nodes, t, no_of_centers):
    distributed_coreset_construction( nodes, t, no_of_centers )
    for v_i in nodes:
        Message_Passing(node.local_coreset,node.neighbors,v_i)
    for v_i in nodes.reverse():
        Message_Passing(node.local_coreset,node.neighbors,v_i)
        node.set_X(clustering_algo( pd.concat(list(node.message_received.values())), no_of_centers ))
    return(node.X)  #will return the centers obtained from last node called in for loop

In [1]:
#Practise cell
#just testing out anything that i feel like adding to the codes above

import pandas as pd
import numpy as np
df = pd.DataFrame({'a':[1,2,3,4,5,6,7],'b':[3,2,3,4,1,4,5]})
df1 = pd.DataFrame({'a':[2,3,4,1,4,6,2],'b':[1,2,4,2,5,6,3]})
#df.index.get_loc(df.head())
l = pd.DataFrame({'a':[1,2,3,4,5,6,7],'b':[3,2,3,4,1,4,5]}).min(axis=1)
k = df.tail(1)
df[np.sqrt(np.square(np.array(df) - np.array(k)).sum(1)) == pd.Series(np.array([0,2,3,4,5 , 7, 0]))]
test_index=pd.Index(list('23154'))
j=[1,2,3,4,5,6,7,8]
1-np.array([j[x] for x in list(map(lambda x:test_index.get_loc(x),pd.Index(['1','2','3'])))])
k=pd.DataFrame()
s={1:df,2:df1}
pd.concat(list(s.values()))
import networkx as nx
j = nx.erdos_renyi_graph(10,0.3)
def f(): 
    global s
    s+=1
s = 0
f()
f()
print(s)
df

2


,a,b
0,1,3
1,2,2
2,3,3
3,4,4
4,5,1
5,6,4
6,7,5


In [14]:
np.array(df.ix[0])

array([1, 3])

In [15]:
np.array(df.ix[1])

array([2, 2])

In [16]:
np.array(df.ix[0])-np.array(df.ix[1])

array([-1,  1])